I recently had a project that required dummy data about user interactions with advertising content presented by an organization.

In order to fake this, I found a data set with a user Id field and plenty records.

From there, I changed some things to personalize it and make it relevant to my project.

This is the process I used.

In [1]:
import numpy as np
import pandas as pd


path = 'C:/Users/alexj/Documents'
file = 'marketing.csv'
filepath = path + '/' + file
data = pd.read_csv(filepath)

#  get rid of irrelevant columns
columns_to_drop = ['date_served', 'date_subscribed', 'date_canceled', 'is_retained', 'converted']
data = data.drop(columns=columns_to_drop)

#  add a column that creates A/B test groups
data['ab_test'] = np.tile(['A', 'B'], len(data) // 2 + 1)[:len(data)]

data.head()
#data.describe()

,user_id,marketing_channel,variant,language_displayed,language_preferred,age_group,subscribing_channel,ab_test
0,a100000029,House Ads,personalization,English,English,0-18 years,House Ads,A
1,a100000030,House Ads,personalization,English,English,19-24 years,House Ads,B
2,a100000031,House Ads,personalization,English,English,24-30 years,House Ads,A
3,a100000032,House Ads,personalization,English,English,30-36 years,House Ads,B
4,a100000033,House Ads,personalization,English,English,36-45 years,House Ads,A


In [2]:
#  create subscribed date
#  creates a random date for each row, between 2018 and now.

num_rows = len(data)

start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2027-01-01')
date_range = (end_date - start_date).days

random_dates = [start_date + pd.Timedelta(days=np.random.randint(0, date_range)) for i in range(num_rows)]
random_dates = [date if date <= pd.to_datetime('now') else pd.NaT for date in random_dates]

data['date_subscribed'] = random_dates

data.head()

,user_id,marketing_channel,variant,language_displayed,language_preferred,age_group,subscribing_channel,ab_test,date_subscribed
0,a100000029,House Ads,personalization,English,English,0-18 years,House Ads,A,2018-06-23
1,a100000030,House Ads,personalization,English,English,19-24 years,House Ads,B,2022-08-10
2,a100000031,House Ads,personalization,English,English,24-30 years,House Ads,A,2022-05-29
3,a100000032,House Ads,personalization,English,English,30-36 years,House Ads,B,2020-09-06
4,a100000033,House Ads,personalization,English,English,36-45 years,House Ads,A,2018-10-24


In [3]:
#  create cancel date (using a function this time)
#  this function will add a column that contains a cancel date some time after the start date, up to 2 years.
#  if the cancel date is after today, it will be null, meaning that they have not cancelled yet.
#  tyhe use of 30 - 731 is based on the number of days in the range.  It can be reste to accommodate specifics.

def add_random_time(date):
    days_to_add = np.random.randint(30, 731)
    new_date = date + pd.DateOffset(days=days_to_add)
    if new_date <= pd.to_datetime('now'):
        return new_date 
    else:
        return pd.NaT

data['date_canceled'] = data['date_subscribed'].apply(add_random_time)

data.head()

,user_id,marketing_channel,variant,language_displayed,language_preferred,age_group,subscribing_channel,ab_test,date_subscribed,date_canceled
0,a100000029,House Ads,personalization,English,English,0-18 years,House Ads,A,2018-06-23,2020-06-14
1,a100000030,House Ads,personalization,English,English,19-24 years,House Ads,B,2022-08-10,2023-10-06
2,a100000031,House Ads,personalization,English,English,24-30 years,House Ads,A,2022-05-29,2024-01-25
3,a100000032,House Ads,personalization,English,English,30-36 years,House Ads,B,2020-09-06,2021-11-17
4,a100000033,House Ads,personalization,English,English,36-45 years,House Ads,A,2018-10-24,2019-09-09


In [4]:
#  adding date of touch
#  date can be any time from 2018 up to cancellation date
#  assumes cancellation implies no more interest

def random_date(row):
    if pd.notnull(row['date_canceled']):
        end_date = row['date_canceled']
    else:
        end_date = pd.to_datetime('now')
    return start_date + pd.to_timedelta(np.random.randint(0, (end_date - start_date).days +1), unit='D')

data['date_of_touch'] = data.apply(random_date, axis=1)

data.head()

,user_id,marketing_channel,variant,language_displayed,language_preferred,age_group,subscribing_channel,ab_test,date_subscribed,date_canceled,date_of_touch
0,a100000029,House Ads,personalization,English,English,0-18 years,House Ads,A,2018-06-23,2020-06-14,2019-03-07
1,a100000030,House Ads,personalization,English,English,19-24 years,House Ads,B,2022-08-10,2023-10-06,2018-06-02
2,a100000031,House Ads,personalization,English,English,24-30 years,House Ads,A,2022-05-29,2024-01-25,2019-07-16
3,a100000032,House Ads,personalization,English,English,30-36 years,House Ads,B,2020-09-06,2021-11-17,2019-02-16
4,a100000033,House Ads,personalization,English,English,36-45 years,House Ads,A,2018-10-24,2019-09-09,2018-04-04


In [5]:
#  add columns for conversion status and retained status

#  make a value to show whether the person was ever converted, and whether converted at time of touch.
data['converted'] = data['date_subscribed'].notnull()
data['converted_at_touch'] = data.apply(lambda row: 
                              pd.notnull(row['date_subscribed']) and row['date_subscribed'] <= row['date_of_touch'], axis=1)
data['converted_counter'] = data['converted'].astype('int')


#  make a value to show whether person is still active
data['retained'] = data['date_canceled'].isnull()
data['retained_counter'] = data['retained'].astype('int')


#  add retnetion length in days
data['retention_length'] = data.apply(lambda row: 
                                    (pd.to_datetime('now') - row['date_subscribed']).days if pd.isnull(row['date_canceled']) 
                                    else (row['date_canceled'] - row['date_subscribed']).days, axis=1)

data.head()

,user_id,marketing_channel,variant,language_displayed,language_preferred,age_group,subscribing_channel,ab_test,date_subscribed,date_canceled,date_of_touch,converted,converted_at_touch,converted_counter,retained,retained_counter,retention_length
0,a100000029,House Ads,personalization,English,English,0-18 years,House Ads,A,2018-06-23,2020-06-14,2019-03-07,True,True,1,False,0,722.0
1,a100000030,House Ads,personalization,English,English,19-24 years,House Ads,B,2022-08-10,2023-10-06,2018-06-02,True,False,1,False,0,422.0
2,a100000031,House Ads,personalization,English,English,24-30 years,House Ads,A,2022-05-29,2024-01-25,2019-07-16,True,False,1,False,0,606.0
3,a100000032,House Ads,personalization,English,English,30-36 years,House Ads,B,2020-09-06,2021-11-17,2019-02-16,True,False,1,False,0,437.0
4,a100000033,House Ads,personalization,English,English,36-45 years,House Ads,A,2018-10-24,2019-09-09,2018-04-04,True,False,1,False,0,320.0


In [6]:
#  adding an extra touch_date.  This date can be any time between 2018 and when they cancelled (or now).  
#  using a lambda function, for the sake of demonstration
#  assumes that cancellations do not come back, for the sake of demonstration.

def random_date_lam(start, end):
    return start + pd.to_timedelta(np.random.randint(0, (end - start).days + 1), unit='D')

data['date_alternative_touch'] = data.apply(lambda row: random_date_lam(start_date,
                                                        row['date_canceled'] if pd.notnull(row['date_canceled']) 
                                                        else pd.to_datetime('now')), axis=1)

data.head()

,user_id,marketing_channel,variant,language_displayed,language_preferred,age_group,subscribing_channel,ab_test,date_subscribed,date_canceled,date_of_touch,converted,converted_at_touch,converted_counter,retained,retained_counter,retention_length,date_alternative_touch
0,a100000029,House Ads,personalization,English,English,0-18 years,House Ads,A,2018-06-23,2020-06-14,2019-03-07,True,True,1,False,0,722.0,2018-12-31
1,a100000030,House Ads,personalization,English,English,19-24 years,House Ads,B,2022-08-10,2023-10-06,2018-06-02,True,False,1,False,0,422.0,2018-04-17
2,a100000031,House Ads,personalization,English,English,24-30 years,House Ads,A,2022-05-29,2024-01-25,2019-07-16,True,False,1,False,0,606.0,2021-09-16
3,a100000032,House Ads,personalization,English,English,30-36 years,House Ads,B,2020-09-06,2021-11-17,2019-02-16,True,False,1,False,0,437.0,2019-09-24
4,a100000033,House Ads,personalization,English,English,36-45 years,House Ads,A,2018-10-24,2019-09-09,2018-04-04,True,False,1,False,0,320.0,2018-01-19


In [7]:
#  change soome channel values to fit the requirements better.

replacements = {
    'House Ads': 'Streaming',
    'Facebook': 'Social',
    'Instagram': 'Influencer'
    }

data[['marketing_channel', 'subscribing_channel']] = data[['marketing_channel', 'subscribing_channel']].replace(replacements)

unique_channels = data['marketing_channel'].unique().tolist()
print(unique_channels)

['Streaming', 'Push', 'Social', 'Influencer', 'Email', nan]


In [8]:
#  change age groups to fit requirements better.

generations = {
    '0-18 years': 'GZ',
    '19-24 years': 'GZ',
    '24-30 years': 'Mil',
    '30-36 years': 'Mil',
    '36-45 years': 'GX',
    '45-55 years': 'GX',
    '55+ years': 'BB'
    }


data['age_group'] = data['age_group'].replace(generations)

unique_channels = data['age_group'].unique().tolist()
print(unique_channels)

['GZ', 'Mil', 'GX', 'BB']


In [9]:
data.to_csv('marketing_dummy_data.csv', index=False)